# Creating and Testing Tools for the FPL Gaffer

## FPL API Data Retrieval Tool

In [1]:
import httpx

In [2]:
base_url = "https://fantasy.premierleague.com/api"
client = httpx.Client()

### Retrieving Bootstrap Data

In [3]:
# Retrieve bootstrap data from FPL API endpoint
bootstrap_endpoint = f"{base_url}/bootstrap-static/"
response = client.get(bootstrap_endpoint)

# Ensure the request was successful
response.raise_for_status()
response_data = response.json()

# Display the keys of the response data
response_data.keys()

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'element_stats', 'element_types', 'elements'])

### Getting Player Data

In [4]:
player_data = response_data["elements"]

# Display the first 10 players
for player in player_data[:10]:
    print(f"ID: {player["id"]}, Name: {player["web_name"]}, Team: {player["team"]}, Position: {player["element_type"]}")

ID: 1, Name: Raya, Team: 1, Position: 1
ID: 2, Name: Arrizabalaga, Team: 1, Position: 1
ID: 3, Name: Hein, Team: 1, Position: 1
ID: 4, Name: Setford, Team: 1, Position: 1
ID: 5, Name: Gabriel, Team: 1, Position: 2
ID: 6, Name: Saliba, Team: 1, Position: 2
ID: 7, Name: Calafiori, Team: 1, Position: 2
ID: 8, Name: J.Timber, Team: 1, Position: 2
ID: 9, Name: Kiwior, Team: 1, Position: 2
ID: 10, Name: Lewis-Skelly, Team: 1, Position: 2


### Getting Gameweek Data

In [7]:
gw_data = response_data.get("events", [])

# Since no gameweek is active now as the season hasn't started,
# let's get the other data than current gameweek

gw_dict = {}
for gw in gw_data:
    # Get the details for the firt gameweek
    gw_dict["gamweek"] = gw.get("id") if gw else None
    gw_dict["deadline"] = gw.get("deadline_time") if gw else None
    gw_dict["finished"] = gw.get("finished") if gw else None
    break

gw_dict

{'gamweek': 1, 'deadline': '2025-08-15T17:30:00Z', 'finished': False}

### Getting Fixture Data

In [11]:
fixture_endpoint = f"{base_url}/fixtures/"

# Get fixture data
fixtures_data = client.get(fixture_endpoint).json()

# Display the first entry from the fixtures data
fixtures_data[0]

{'code': 2561895,
 'event': 1,
 'finished': False,
 'finished_provisional': False,
 'id': 1,
 'kickoff_time': '2025-08-15T19:00:00Z',
 'minutes': 0,
 'provisional_start_time': False,
 'started': False,
 'team_a': 4,
 'team_a_score': None,
 'team_h': 12,
 'team_h_score': None,
 'stats': [],
 'team_h_difficulty': 3,
 'team_a_difficulty': 5,
 'pulse_id': 124791}

## FPL News Searcher Tool

In [13]:
from dotenv import load_dotenv, find_dotenv
import os
from langchain_community.tools import TavilySearchResults

In [14]:
api_key = os.getenv("TAVILY_API_KEY")

In [15]:
# Load environment variables
_ = load_dotenv()

# Create tavily search tool instance
tavily = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
)

In [16]:
tavily.invoke({"query": "Premier league team news press conference"})

[{'title': 'Premier League Football - Latest news, results, stats & transfers - BBC',
  'url': 'https://www.bbc.com/sport/football/premier-league',
  'content': "Stephen: Hopefully, this shows we don't have to sell our best players to survive financially. This is a massive statement of intent. He is a key player that we need to keep and continue to develop. Future captain!\n\n   Tom: I'm very glad we kept a talent like Jarrad - he is one of my favourite players in this Everton team and brings confidence to the defence. He drives the team on and on, and I think he should be captain when Seamus Coleman leaves. [...] Andy: Calvert-Lewin has done a job for the last few seasons but if we're going to escape the relegation battles, we need to find a striker who will deliver more and push us up the table. Shake hands and wish him the best.\n\n    Chris: It's a shame we couldn't hold on to him for one more season as a squad player with a generous contract until we could get a new striker. [...]